In [ ]:
!pip install duckdb pandas --quiet

In [1]:
import json
import sys
import duckdb
import pandas as pd
from pathlib import Path
from google.cloud import bigquery

In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=dOwXAuNBbLhneLZAy97wTVGya9FOcl&access_type=offline&code_challenge=pGBA7NakFPyoEEelJsejVs_i6bzFauJ2-M5JJbAdAyE&code_challenge_method=S256


Credentials saved to file: [/Users/edshow/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "site-hypr" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some Google Cloud CLI compon

In [6]:
bq_client = bigquery.Client()

In [7]:
duckdb_conn = duckdb.connect(database=':memory:')

In [8]:
base_dirs = [
    '/Users/edshow/Downloads/tokens_sat_20250704_0-responses-20250707-1-tdfai37',
    '/Users/edshow/Downloads/tokens_sat_20250704_1-responses-20250707-1-8gsaoe',
    '/Users/edshow/Downloads/tokens_sat_20250704_2-responses-20250707-1-wim0bw',
    '/Users/edshow/Downloads/tokens_sat_20250704_3-responses-20250707-1-m3smkk',
]

In [9]:
all_json_files = [
    str(p) for base in base_dirs for p in Path(base).glob("*.json")
]

In [9]:
nfe_dfs = []
for file in nfe_file_list:
    tmp = pd.read_json(file, lines=True)
    tmp['token'] = tmp['header'].apply(lambda x: x['parameters']['nfce'])
    nfe_dfs.append(tmp)

In [10]:
sat_dfs = []
for file in all_json_files:
    tmp = pd.read_json(file, lines=True)
    tmp['token'] = tmp['header'].apply(lambda x: x['parameters']['chave'])
    sat_dfs.append(tmp)

In [11]:
agg_nfe_df = pd.concat(nfe_dfs, ignore_index=True)
agg_sat_df = pd.concat(sat_dfs, ignore_index=True)
agg_df = pd.concat([agg_nfe_df, agg_sat_df], ignore_index=True)
agg_df['full_json'] = agg_df.apply(lambda row: json.dumps(row.to_dict(), ensure_ascii=False), axis=1)
agg_df = agg_df[['token', 'full_json']]
agg_df.columns = ['token', 'data']
agg_df.head()


NameError: name 'nfe_dfs' is not defined

In [12]:
agg_sat_df = pd.concat(sat_dfs, ignore_index=True)
agg_sat_df['full_json'] =agg_sat_df.apply(lambda row: json.dumps(row.to_dict(), ensure_ascii=False), axis=1)
agg_sat_df = agg_sat_df[['token', 'full_json']]
agg_sat_df.columns = ['token', 'data']
agg_sat_df.head()


,token,data
0,35250442591651032428590006754311646667978510,"{""code"": 200, ""code_message"": ""A requisição fo..."
1,35250342591651256458590012126590353664220000,"{""code"": 605, ""code_message"": ""A consulta não ..."
2,35250342591651090460590011211571371966937950,"{""code"": 605, ""code_message"": ""A consulta não ..."
3,35250261412110045841650530000497301824357569,"{""code"": 608, ""code_message"": ""Os parâmetros f..."
4,35250342591651019405590014550610208542393699,"{""code"": 605, ""code_message"": ""A consulta não ..."


In [17]:
tmp = agg_sat_df.copy()
total = tmp["token"].nunique()
tmp["code"] =tmp["data"].apply(lambda x: json.loads(x)["code"])
tmp = tmp.groupby("code")["token"].nunique().reset_index(name="distinct_token_count")
tmp["percent"] = (tmp["distinct_token_count"] / total * 100).round(2)
tmp

,code,distinct_token_count,percent
0,200,49234,41.25
1,605,51710,43.32
2,608,18421,15.43


In [25]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pytz


agg_df = agg_sat_df.copy()

rows_to_insert = []

for row in agg_df.itertuples():
    try:
        rows_to_insert.append({
            "token": row.token,
            "moment": datetime.now(pytz.timezone('America/Sao_Paulo')).isoformat(),
            "data": row.data,
            "status": 'PROCESSADO'
        })
    except Exception as e:
        print(f"Error for token {row.token}: {e}")


In [26]:
BATCH_SIZE = 1000
for i in range(0, len(rows_to_insert), BATCH_SIZE):
    batch = rows_to_insert[i:i+BATCH_SIZE]
    errors = bq_client.insert_rows_json('cashify-sx03a5.pubsub_messages.nfeasy_tokens_data', batch)
    if errors:
        print(f"Errors in batch {i//BATCH_SIZE}: {errors}")